In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#anaconda env notebook has modified python source
import numpy as np
import pandas as pd
import glob
import sys
sys.path.insert(0,'/wrk/simon/scripts/python/functions/')
from read_teds9 import teds9_df,source_type,teds9_smoke0,gen_header,yulin_emis
%load_ext autoreload

In [9]:
ddir='/archives/data/tw/teds9/'
files=glob.glob(''.join([ddir,'[!b]*[!m].dbf']))
gen_orl=0
gen_toxorl=0
gen_srcgrp=0

TEDS 9.0
----------
----

In [3]:
#files=files[0:2]
files

['/archives/data/tw/teds9/linegrid.dbf',
 '/archives/data/tw/teds9/areagrid.dbf',
 '/archives/data/tw/teds9/point.dbf']

Conversion to ORL point source format
-----------
---------

In [4]:
pref='pt'
item='inv'
SRCABBR0='point'
ddir1=''.join(['/archives/utils/SMOKE/data/inventory/teds9/',SRCABBR0,'/'])
#ddir1=''.join(['/archives/utils/SMOKE/data/inventory/teds9_cb5',SRCABBR0,'/'])
lfile=''.join([ddir1,pref,item,'.',SRCABBR0,'.lst'])

In [8]:
if gen_orl==1:
    f1=open(lfile,'w')
    f1.write('#LIST\n')
    f1.close()
    for n,file in enumerate(files):
        df=teds9_df(file)
        ftype,stype,SRCABBR,pref,suf=source_type(file)
        ofile=''.join([ddir1,pref,item,'_',suf])
        print(''.join([file,'; ',ofile]))
        f1=open(lfile,'a')
        f1.write(''.join([ofile,'\n']))
        f1.close()
        gen_header(ftype,stype,ofile)
        teds9_smoke0(SRCABBR,df,ofile)

Emission records of toxic pollutant in worst case senario(using mininmun stack heights)
------------
----------

In [6]:
#gen_toxrol=2

In [7]:
#read and output addition yulin base emission
if gen_toxorl==1:
    #all in 1 file
    file=files[2]
    ftype,stype,SRCABBR,pref,suf=source_type(file)
    ddir1=''.join(['/archives/utils/SMOKE/data/inventory/teds9/',SRCABBR0,'/'])
    ofile=''.join([ddir1,pref,item,'_yulin_',suf])
    gen_header(ftype,stype,ofile)
    yulin_emis(ofile,crit='worst')
    lfile=''.join([ddir1,pref,item,'.',SRCABBR0,'.lst'])
    f1=open(lfile,'a')
    f1.write(''.join([ofile,'\n']))
    f1.close()
elif gen_toxrol==2:
    #1 file for each stack
    yulin_emis('tmp',crit='worst',grp='each')

In [ ]:
#source groups(total)
if gen_srcgrp==0:    
    efile='/archives/data/tw/teds9/point.dbf'
    dfile='/archives/data/tw/teds9/emissionLoc.xlsx'
    df=teds9_df(efile)
    df1=pd.read_excel(dfile,dtype=np.str,sheetname='Y2014',names=['factory','shp','VINYCHLRT','BUTADIE','PROPDICLR','BENZENE','TRICLA112','MECL','TTCLE1122','ETHDIBROM','CHCL3','CARBONTET','CL2_C2_12','ETHYLBENZ','STYRENE','CUMENE','ISOP','DICHLRBNZN','key'])
    df1['SERIAL_NO']=df1.index.map(lambda x:'YL{:04d}'.format(int(x))).values
    lfile='/archives/utils/SMOKE/data/inventory/teds9/other/source_groups.nc.txt'
    f1=open(lfile,'w')
    for nn,row in df1.iterrows():
        f1.write(''.join([str(nn+1),' 739130 000000 ',row['SERIAL_NO'],' \n']))                     
    f1.close()

In [ ]:
#source groups(stacks)
if gen_srcgrp==0:    
    efile='/archives/data/tw/teds9/point.dbf'
    dfile='/archives/data/tw/teds9/emissionLoc.xlsx'
    df=teds9_df(efile)
    df1=pd.read_excel(dfile,dtype=np.str,sheetname='Y2014',names=['factory','shp','VINYCHLRT','BUTADIE','PROPDICLR','BENZENE','TRICLA112','MECL','TTCLE1122','ETHDIBROM','CHCL3','CARBONTET','CL2_C2_12','ETHYLBENZ','STYRENE','CUMENE','ISOP','DICHLRBNZN','key'])
    df1['SERIAL_NO']=df1.index.map(lambda x:'YL{:04d}'.format(int(x))).values
    lfile='/archives/utils/SMOKE/data/inventory/teds9/other/source_groups.nc.txt'
    f1=open(lfile,'w')
    for nn,row in df1.iterrows():
        f1.write(''.join([str(nn+1),' 739130 000000 ',row['SERIAL_NO'],' \n']))                     
    f1.close()

Machine Learning for observation based emission estimation
------
------
$$C_j=\sum^{n'}_{i=0}\theta(\varepsilon_i,l_j,t)$$
$\theta(\varepsilon_i,l_j,t)$ is the contribution of the emission($\varepsilon_i$ at time t) of a pollutant to the location $l_j$ at time t. Sum of $\theta$ from all emissions($n'$) is the pollutant concentration($C_j$) at location $l_j$.
CMAQ DDM 3D sensitivity(S)
-----------
$$S_{ij}=\frac{\partial C_j}{\partial \varepsilon_i}$$
$$\therefore C_j=\sum^{n'}_{i=1}\varepsilon_iS_{ij}$$
Model the observed pollutant concetration as(surrogate model)
$$\left[\begin{array}{c}
C_1(t)  \\
C_2(t)  \\
\vdots \\
C_{m-1}(t)  \\
C_m(t)
\end{array}\right] \approx
\left[\begin{array}{c}
H_1(t)  \\
H_2(t)  \\
\vdots \\
H_{m-1}(t)  \\
H_m(t)
\end{array}\right]=
\left[\begin{array}{cccc}
\sigma_1(t) & S_1(t)\varepsilon_1(t) & S_1(t)\varepsilon_2(t) & \cdots & S_1(t)\varepsilon_n(t) \\
\sigma_2(t) & S_2(t)\varepsilon_1(t) & S_2(t)\varepsilon_2(t) & \cdots & S_2(t)\varepsilon_n(t) \\
\vdots & \vdots & \vdots & \ddots & \vdots  \\
\sigma_{m-1}(t) & S_{m-1}(t)\varepsilon_1(t) & S_{m-1}(t)\varepsilon_2(t) & \cdots & S_{m-1}(t)\varepsilon_n(t) \\
\sigma_m(t) & S_{m}(t)\varepsilon_1(t) & S_{m}(t)\varepsilon_2(t) & \cdots & S_{m}(t)\varepsilon_n(t) 
\end{array}\right]
\left[\begin{array}{c}
x_0(t) \\
x_1(t)  \\
x_2(t)  \\
\vdots \\
x_{n-1}(t)  \\
x_n(t)
\end{array}\right]
$$
$\sigma_1 \cdots \sigma_m$ are the unknown at location 1 $\cdots$ m. The unknown includes model error and pollutant concentration cannot be explained by the emssions $\varepsilon_1 \cdots \varepsilon_n(n<n')$. $x_1 \cdots x_n$ are the modification to these emissions so the hypothesis($H_1 \cdots H_m$) are closer to observed concentration($C_1 \cdots C_m$).  
The goal is finding the best modification to minimise the difference between the observed with model results. That is minimise the cost function J.
$$J=\frac{1}{2}\sum^m_{j=1}(H_j-C_j)^2$$
There maybe no sufficient data to derive $x_i$ including its temperol pattern. For instance, observed concentration($C_j$) for a pollutant species lack representation in time.

Addtional input
--------
Include boundary condition as another factor

Attempt I(linear regression with machine learning)
-------
Let $ x_i \ \& \ \varepsilon_i $ invariant over time for each stack and assuming emission profile or partial profile of earch stack is correct. $C_j$ can then expanded to different pollutant species. $C_j$ location subscript $1 \cdots m$ is then a combination of profile species($k \subset 1 \cdots o$) and location($C_{jk}$).
and
$$C_{jk}=\sum^{n'}_{i=1}x_{ik}\varepsilon_{ik}S_{ijk}$$
$$J=\frac{1}{2}\sum^{m,o}_{j=1,k=1}(H_{jk}(t)-C_{jk}(t))^2$$

Attempt II(Extended Kalman filter)
----------
Let $ x_i \ \& \ \varepsilon_i $ invariant over time for each stack and assuming emission profile or partial profile of earch stack is correct. $C_j$ can then expanded to different pollutant species. $C_j$ location subscript $1 \cdots m$ is then a combination of profile species($k \subset 1 \cdots o$) and location($C_{jk}$). $v_{jk}$ is a noise term which defined as the difference between $C_{jk}$ and $H_{jk}$.  
$$C_{jk}(t)=S H_{jk}(t)+v_{jk}(t)$$
$$H_{jk}(t)=S_1 H_{jk}(t-1)+S_0 u_{jk}(t)$$

Furthermore, define Kalman gain($g_{jk}$) as
$$\hat{H_{jk}}(t)=\hat{H_{jk}}(t-1)+g_{jk}(t)(C_{jk}-\hat{H_{jk}}(t-1))$$
$$g_{jk}(t)=\frac{\sigma_{jk}(t)}{\sigma_{jk}(t-1)+r}$$

Predicting next time step
$$\hat{H_{jk}}(t)=S_1 \hat{H_{jk}}(t-1)+S_0 u_{jk}$$
$$\sigma_{jk}(t)=S_1\sigma_{jk}(t-1)S_1$$
$$g_{jk}(t)=\frac{S\sigma_{jk}(t)}{S\sigma_{jk}(t-1)S+r}$$
$$\hat{H_{jk}}(t)\leftarrow\hat{H_{jk}}(t)+g_{jk}(t)(C_{jk}(t)-S\hat{H_{jk}}(t))$$
$$\sigma_{jk}(t)\leftarrow(1-Sg_{jk})\sigma_{jk}$$

Run CMAQ in 6 hours interval and update each 6 hourly result with Kalman filter then use the updated last time step as initial condition for the next 6 hours run.

Attempt III(pySOT)
-------
Derive a surrogate using pySOT. Input parameters includes all cmaq input and results from pervious time step. Steps pending.

To-DO
--------
1.Derive a temporal profile for the stacks  
2.Add sensitivity of boundary condition  
3.Add photolysis and reaction to the species added(may not needed as there is no photolysis rate for benzene and isoprene in the code)   
4.Add large scale emission from EDGER database(sensitivity of BC may sufficient to replace this)

In [ ]:
from read_teds9 import teds_att
efile='/archives/data/tw/teds9/point.dbf'
dfile='/archives/data/tw/teds9/emissionLoc.xlsx'
df=teds9_df(efile)
df1=pd.read_excel(dfile,dtype=np.str,sheetname='Y2014',names=['factory','shp','VINYCHLRT','BUTADIE','PROPDICLR','BENZENE','TRICLA112','MECL','TTCLE1122','ETHDIBROM','CHCL3','CARBONTET','CL2_C2_12','ETHYLBENZ','STYRENE','CUMENE','ISOP','DICHLRBNZN','key'])
df1['SERIAL_NO']=df1.index.map(lambda x:'YL{:04d}'.format(int(x))).values
df0=df1['key'].map(lambda x:teds_att(df,x,crit='worst'))
df1['WGS84_E']=df0.map(lambda x:x[0]).values
df1['WGS84_N']=df0.map(lambda x:x[1]).values

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np

map = Basemap(projection='merc' , 
            resolution='i' , fix_aspect=True,
            llcrnrlon=120.0 , llcrnrlat=23.5, 
            urcrnrlon=120.5 , urcrnrlat=24.5,
            lat_ts =20)
fcx,fcy=map(df1['WGS84_E'].values, df1['WGS84_N'].values)
#海岸線的寬度
map.drawcoastlines(linewidth=1)

#畫緯度線23.5和25,labels標示出座標度數,fontsize文字大小
map.drawparallels(np.arange(23.5 , 24) , labels=[1,0,0,0] , fontsize=10)

#畫經度線120和122,labels標示出座標度數,fontsize文字大小
map.drawmeridians(np.arange(120,122),labels=[0,0,0,1],fontsize=10)

plt.plot(fcx,fcy,'ro', markersize = 2)
#map 底色backgroubd-color
#map.drawmapboundary(fill_color='aqua')

#取一個title
plt.title('Practice')

#設定XY說明label , xy範圍0~1(預設是中間0.5)
plt.xlabel('lon' , fontsize=12 , x=0.5)
plt.ylabel('lat' , fontsize=12 , y=0.5)

plt.show()



In [ ]:
df2 = pd.DataFrame()
df2 = df2.append(df1['WGS84_E'], ignore_index = True)
df2 = df2.append(df1['WGS84_N'], ignore_index = True)
df3 = df2.T
df3.to_csv('aab.csv')

In [ ]:
df1